<a href="https://colab.research.google.com/github/Angie-ch/waste-classification-/blob/main/waste_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
#import tensorflow.contrib.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report,confusion_matrix
from PIL import Image
from pathlib import Path
import scipy
import os
import numpy as np
import matplotlib.pyplot as plt
from torchvision.datasets import ImageFolder
import torchvision.transforms as T

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# collect directory
data_dir = Path('Garbage\original_images')

transformer = T.Compose([T.Resize((32, 32)), T.ToTensor()])
dataset = ImageFolder(data_dir, transform = transformer)

# display class names
print(dataset.classes)

In [23]:
import os
import shutil
import random

# Define the path to the dataset
dataset_dir = '/content/drive/MyDrive/waste dataset'

# Define the path to where the split datasets will be saved
train_dir = 'train_data'
test_dir = 'test_data'

# List of class names
class_names = ['glass', 'plastic', 'paper', 'metal']

# Create directories for train and test sets (if they do not exist)
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Create subdirectories for each class in train and test directories (if they do not exist)
for class_name in class_names:
    os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)
    os.makedirs(os.path.join(test_dir, class_name), exist_ok=True)

# Split the dataset for each class
for class_name in class_names:
    # Get the path to the subfolder inside each class
    class_folder = os.path.join(dataset_dir, class_name)

    # List subdirectories (e.g., "paper waste", "glass waste")
    subfolders = [f for f in os.listdir(class_folder) if os.path.isdir(os.path.join(class_folder, f))]

    for subfolder in subfolders:
        subfolder_path = os.path.join(class_folder, subfolder)

        # Get the list of image files in the current subfolder
        image_files = [f for f in os.listdir(subfolder_path) if os.path.isfile(os.path.join(subfolder_path, f))]

        # Shuffle the list of image files to ensure randomness
        random.shuffle(image_files)

        # Calculate the number of images for training (70%)
        total_images = len(image_files)
        train_size = int(0.7 * total_images)

        # Split the files into training and testing sets
        train_images = image_files[:train_size]
        test_images = image_files[train_size:]

        # Move the training images to the train directory
        for image in train_images:
            src = os.path.join(subfolder_path, image)
            dst = os.path.join(train_dir, class_name, image)
            shutil.move(src, dst)

        # Move the testing images to the test directory
        for image in test_images:
            src = os.path.join(subfolder_path, image)
            dst = os.path.join(test_dir, class_name, image)
            shutil.move(src, dst)

        print(f"Processed subfolder '{subfolder}' of class '{class_name}': {len(train_images)} train images, {len(test_images)} test images")

print("Dataset split complete!")


Processed subfolder 'glass waste' of class 'glass': 0 train images, 0 test images
Processed subfolder '.ipynb_checkpoints' of class 'glass': 0 train images, 0 test images
Processed subfolder 'Plastic' of class 'plastic': 0 train images, 0 test images
Processed subfolder 'paper waste' of class 'paper': 0 train images, 0 test images
Processed subfolder 'metal waste' of class 'metal': 0 train images, 0 test images
Processed subfolder '.ipynb_checkpoints' of class 'metal': 0 train images, 0 test images
Dataset split complete!


In [3]:
PATH_TRAIN = r"content\drive\myDrive\train_data"
PATH_TEST = r"content\drive\myDrive\test_data"
class_names = ['glass', 'metal','paper','plastic']

In [18]:
import os
from PIL import Image

# Define image paths using os.path.join
imagepath_paper = os.path.join('train_data', 'paper')
graypath_paper = os.path.join('test_data', 'paper')

# Ensure the output directory exists
os.makedirs(graypath_paper, exist_ok=True)
# Check if the directory exists
os.path.exists(imagepath_paper)  # This should return True if the path is correct

print(f"Checking path: {imagepath_paper}")
if os.path.exists(imagepath_paper):
    print("Directory found!")
    file_listing = os.listdir(imagepath_paper)
else:
    print(f"Directory not found: {imagepath_paper}")

# List files in the source directory
file_listing = os.listdir(imagepath_paper)

# Process each file in the directory
for file in file_listing:
    # Build the full path to the image file
    file_path = os.path.join(imagepath_paper, file)

    im = Image.open(file_path)

    img = im.resize((32, 32))

    # Convert the image to grayscale
    gray = img.convert('L')

    gray_file_path = os.path.join(graypath_paper, file)

    # Save the grayscale image as JPEG
    gray.save(gray_file_path, "JPEG")

print("Image processing completed successfully.")


Checking path: train_data/paper
Directory found!
Image processing completed successfully.


In [ ]:
#for training
imagepath_paper = r"content\drive\myDrive\train_data\paper"
#for testing
graypath_paper= r"content\drive\myDrive\test_data\paper"
File_listing = os.listdir(imagepath_paper)
for file in File_listing:
    im = Image.open(imagepath_paper + '\\' + file)
    img = im.resize((32,32))
    gray = img.convert('L')
    gray.save(graypath_paper + '\\' + file, "JPEG")
